# Black-Litterman Portfolio Optimization - Quick Start

This notebook demonstrates a basic workflow for using the Black-Litterman model for portfolio optimization.

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from src import data_loader
from src import factors
from src import covariance
from src import black_litterman
from src import optimization
from src import visualization

%matplotlib inline

## Step 1: Load Data

Load all required data including ETF prices, factor data, and CMA priors.

In [ ]:
# Load all data
data = data_loader.load_all_data(data_dir='../data')

print("Available data:")
for key in data.keys():
    if isinstance(data[key], pd.DataFrame) or isinstance(data[key], pd.Series):
        print(f"  {key}: Shape {data[key].shape}")

# Extract key components
returns = data['returns_monthly']
excess_returns = data['excess_returns']
cma_priors = data['cma_priors']

print("\nAssets in universe:")
print(returns.columns.tolist())

## Step 2: Build Factor Model

Construct custom factors and estimate factor exposures.

In [ ]:
# Build custom factors
custom_factors = factors.build_custom_factors(returns)

# Combine with Fama-French factors
ff_factors = data['ff_factors']['FF5']
all_factors = factors.combine_factors(ff_factors, custom_factors)

# Estimate factor model
factor_results = factors.factor_model_analysis(
    excess_returns,
    ff_factors,
    custom_factors
)

print("Factor Model R-squared:")
print(factor_results['r_squared'].sort_values(ascending=False))

## Step 3: Estimate Covariance Matrix

Use Ledoit-Wolf shrinkage for robust covariance estimation.

In [ ]:
# Estimate covariance using Ledoit-Wolf shrinkage
cov_matrix, shrinkage = covariance.ledoit_wolf_shrinkage(excess_returns)

print(f"Shrinkage intensity: {shrinkage:.3f}")

# Visualize correlation matrix
fig = visualization.plot_correlation_matrix(
    excess_returns,
    title="Asset Correlation Matrix"
)
plt.show()

## Step 4: Black-Litterman Model

Compute posterior expected returns incorporating views.

In [ ]:
# Initialize Black-Litterman model with CMA priors
bl_model = black_litterman.BlackLittermanModel(
    prior_returns=cma_priors,
    cov_matrix=cov_matrix,
    tau=0.025
)

# Add views (example)
views = {
    "SPY": 0.10,  # US equities will return 10%
    "VWO": 0.12,  # EM will return 12%
}

bl_model.add_absolute_views(views, confidences={"SPY": 0.7, "VWO": 0.6})

# Compute posterior
posterior_returns, posterior_cov = bl_model.compute_posterior()

# Compare prior and posterior
comparison = bl_model.get_view_deviations()
print("\nPrior vs Posterior Expected Returns:")
print(comparison[['Prior', 'Posterior', 'Deviation']].round(4))

## Step 5: Portfolio Optimization

Optimize portfolio weights using posterior expected returns.

In [ ]:
# Mean-variance optimization
weights_mv = optimization.mean_variance_optimization(
    expected_returns=posterior_returns,
    cov_matrix=cov_matrix,
    risk_aversion=2.0,
    constraints={"long_only": True, "max_weight": 0.4}
)

# Maximum Sharpe ratio
weights_sharpe = optimization.max_sharpe_ratio(
    expected_returns=posterior_returns,
    cov_matrix=cov_matrix,
    constraints={"long_only": True}
)

# Risk parity
weights_rp = optimization.risk_parity_portfolio(cov_matrix)

# Compare portfolios
all_weights = pd.DataFrame({
    "Mean-Variance": weights_mv,
    "Max Sharpe": weights_sharpe,
    "Risk Parity": weights_rp
})

print("\nOptimal Portfolio Weights:")
print(all_weights.round(3))

## Step 6: Visualize Results

In [ ]:
# Plot portfolio weights
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, col in enumerate(all_weights.columns):
    all_weights[col].sort_values().plot(
        kind='barh',
        ax=axes[i],
        color='steelblue',
        title=col
    )
    axes[i].set_xlabel('Weight')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Portfolio statistics
print("\nPortfolio Statistics (Annualized):")
print("=" * 60)

for name, weights in zip(all_weights.columns, [weights_mv, weights_sharpe, weights_rp]):
    stats = optimization.portfolio_statistics(
        weights,
        posterior_returns,
        cov_matrix
    )
    print(f"\n{name}:")
    print(f"  Expected Return: {stats['annual_return']:.2%}")
    print(f"  Volatility:      {stats['annual_volatility']:.2%}")
    print(f"  Sharpe Ratio:    {stats['annual_sharpe']:.3f}")

## Step 7: Efficient Frontier

In [ ]:
# Compute efficient frontier
frontier_returns, frontier_vols, _ = optimization.efficient_frontier(
    posterior_returns,
    cov_matrix,
    n_points=50
)

# Plot efficient frontier with portfolios
portfolio_stats = []
for weights in [weights_mv, weights_sharpe, weights_rp]:
    stats = optimization.portfolio_statistics(weights, posterior_returns, cov_matrix)
    portfolio_stats.append(stats)

portfolio_returns = pd.Series([s['return'] for s in portfolio_stats])
portfolio_vols = pd.Series([s['volatility'] for s in portfolio_stats])

fig = visualization.plot_efficient_frontier(
    frontier_returns,
    frontier_vols,
    portfolio_returns,
    portfolio_vols,
    all_weights.columns.tolist()
)
plt.show()

## Conclusion

This notebook demonstrated:
1. Loading and preparing data
2. Building factor models
3. Estimating robust covariance matrices
4. Incorporating views with Black-Litterman
5. Optimizing portfolio weights
6. Analyzing and visualizing results

For more advanced analysis, see the `main_analysis.ipynb` notebook.